In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from utils import (
    agregar_meses_segun_inflacion,
    show_plot,
)

In [ ]:
def set_bias(last_check: datetime) -> int:
    current_date = datetime.now()
    years_diff = current_date.year - last_check.year
    months_diff = current_date.month - last_check.month
    return years_diff * 12 + months_diff

In [ ]:
last_check = datetime(
    2025, 2, 1
)  # ACTUALIZAR CUANDO SE AGREGUEN DATOS FUTUROS A ESTA FECHA Y REVISAR TODOS LOS DATOS
MONTHS_TO_UPDATE = set_bias(last_check)
MONTHS_TO_UPDATE

In [ ]:
datos_personales = pd.read_csv("../data/sueldos.csv")

In [ ]:
datos_personales["sueldo_bruto_usd_oficial"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_bruto_usd_blue"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_blue"]
)
datos_personales["sueldo_neto_usd_oficial"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_neto_usd_blue"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_blue"]
)
datos_personales.tail()

In [ ]:
### USAR PARA ACTUALIZAR MANUALMENTE EL CSV DE VUELOS

# vuelos = pd.read_csv("../data/vuelos.csv")
# vuelos = agregar_meses_segun_inflacion(vuelos, MONTHS_TO_UPDATE, "precio_en_ars")
# vuelos

In [ ]:
vuelos = pd.read_csv("../data/vuelos.csv")
vuelos["fecha"] = pd.to_datetime(vuelos["fecha"])
vuelos.set_index("fecha", inplace=True)
monthly_vuelos = vuelos[["precio_en_ars"]].resample("M").mean()
monthly_vuelos["precio_en_ars"] = monthly_vuelos["precio_en_ars"].interpolate()
monthly_vuelos["destino"] = vuelos["destino"].resample("M").ffill()
monthly_vuelos.tail(5)

In [ ]:
monthly_vuelos = monthly_vuelos[-len(datos_personales) :]
ratio = (
    monthly_vuelos["precio_en_ars"].values / datos_personales["sueldo_neto_ars"].values
) * 100
labels = datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str)
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    labels,
    ratio,
    label="% del sueldo neto necesario para vuelo",
    color="purple",
    marker="o",
)

for i, val in enumerate(ratio):
    plt.text(i, val + 1, f"{val:.1f}%", ha="center", fontsize=8)

plt.axhline(y=100, color="blue", linestyle="--", linewidth=1, label="100% del sueldo")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()

show_plot(
    plot_id,
    "Mes",
    "Porcentaje del sueldo neto (%)",
    "Evolución del costo de vuelos a EEUU como % del Sueldo Neto",
)

In [ ]:
# Precio promedio de publicación (pesos) de departamentos en alquiler de 1, 2 y 3 ambientes usados y a estrenar

alquileres = pd.read_excel(
    "https://www.estadisticaciudad.gob.ar/eyc/wp-content/uploads/2025/04/MI_DAP_AX15.xlsx",
    header=2,
    skipfooter=3,
)
alquileres.tail(6)

In [ ]:
alquileres.columns = ["año", "mes", "precio_1_amb", "precio_2_amb", "precio_3_amb"]
del alquileres["precio_2_amb"]
del alquileres["precio_3_amb"]
alquileres["año"] = alquileres["año"].ffill()
alquileres["año"] = alquileres["año"].astype(int)
alquileres["precio_1_amb"] *= 0.7
alquileres = agregar_meses_segun_inflacion(
    alquileres, MONTHS_TO_UPDATE - 3, "precio_1_amb", "mes"
)
alquileres = alquileres[-len(datos_personales) :]
alquileres.tail(6)

In [ ]:
alquileres = alquileres[-len(datos_personales) :]

ratio = (
    alquileres["precio_1_amb"].values / datos_personales["sueldo_neto_ars"].values
) * 100

labels = datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str)

plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    labels,
    ratio,
    label="% del sueldo neto necesario para alquiler 1 ambiente",
    color="darkgreen",
    marker="o",
)

for i, val in enumerate(ratio):
    plt.text(i, val + 1, f"{val:.1f}%", ha="center", fontsize=8)

plt.axhline(y=30, color="blue", linestyle="--", linewidth=1, label="30% del sueldo")

plt.xticks(rotation=45)
plt.grid(True)
plt.legend()

show_plot(
    plot_id,
    "Mes",
    "Porcentaje del sueldo neto (%)",
    "Evolución del costo de alquiler 1 ambiente como % del Sueldo Neto",
)


In [ ]:
autos = pd.read_csv("../data/autos.csv")
# autos = agregar_meses_segun_inflacion(autos, MONTHS_TO_UPDATE-4, "ars_promedio", "fecha")  # UTILIZAR SI HACE FALTA ACTUALIZAR TEMPORALMENTE
autos

In [ ]:
autos = autos[-len(datos_personales) :]

ratio = autos["ars_promedio"].values / datos_personales["sueldo_neto_ars"].values

labels = datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str)

plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    labels,
    ratio,
    label="N° de sueldos netos necesarios para auto promedio",
    color="darkred",
    marker="o",
)

for i, val in enumerate(ratio):
    plt.text(i, val + 1, f"{val:.1f}%", ha="center", fontsize=8)

plt.axhline(y=8, color="blue", linestyle="--", linewidth=1, label="24 cuotas del 33% del sueldo")

plt.xticks(rotation=45)
plt.grid(True)
plt.legend()

show_plot(
    plot_id,
    "Mes",
    "Número de sueldos netos",
    "Evolución del costo de un auto promedio como N° de Sueldos Netos",
)


In [ ]:
big_mac = pd.read_csv(
    "https://github.com/TheEconomist/big-mac-data/releases/download/2024-07/big-mac-full-index.csv"
)
big_mac = big_mac[big_mac["iso_a3"] == "ARG"][["date", "local_price"]]
big_mac["date"] = pd.to_datetime(big_mac["date"])
big_mac.set_index("date", inplace=True)
big_mac = big_mac.resample("M").mean()
big_mac["local_price"] = big_mac["local_price"].interpolate()
big_mac = agregar_meses_segun_inflacion(big_mac, MONTHS_TO_UPDATE + 7, "local_price")
big_mac = big_mac[-len(datos_personales) :]
big_mac["local_price"] = big_mac["local_price"] * 60
big_mac.tail(10)

In [ ]:
big_mac = big_mac[-len(datos_personales) :]
ratio = (big_mac["local_price"].values * 60) / datos_personales[
    "sueldo_neto_ars"
].values

labels = datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str)

plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    labels,
    ratio,
    label="% del sueldo neto necesario para 60 Big Macs",
    color="orange",
    marker="o",
)

for i, val in enumerate(ratio):
    plt.text(i, val + 1, f"{val:.1f}%", ha="center", fontsize=8)

plt.axhline(y=20, color="blue", linestyle="--", linewidth=1, label="20% del sueldo")

plt.xticks(rotation=45)
plt.grid(True)
plt.legend()

show_plot(
    plot_id,
    "Mes",
    "Porcentaje del sueldo neto (%)",
    "Evolución del valor de 60 Big Macs como % del Sueldo Neto",
)


In [ ]:
variacion_sueldo_neto_ars = (
    datos_personales["sueldo_neto_ars"].iloc[-1] * 100
) / datos_personales["sueldo_neto_ars"].iloc[0]
variacion_alquileres = (alquileres["precio_1_amb"].iloc[-1] * 100) / alquileres[
    "precio_1_amb"
].iloc[0]
variacion_big_mac = (big_mac["local_price"].iloc[-1] * 100) / big_mac[
    "local_price"
].iloc[0]
variacion_autos = (autos["ars_promedio"].iloc[-1] * 100) / autos["ars_promedio"].iloc[0]
variacion_monthly_vuelos = (
    monthly_vuelos["precio_en_ars"].iloc[-1] * 100
) / monthly_vuelos["precio_en_ars"].iloc[0]

variacion_inflacion = 100
for index in range(1, len(datos_personales["inflacion_del_mes_anterior"])):
    variacion_inflacion *= 1 + (
        datos_personales["inflacion_del_mes_anterior"][index] / 100
    )

variaciones = [
    variacion_sueldo_neto_ars - 100,
    variacion_inflacion - 100,
    variacion_alquileres - 100,
    variacion_big_mac - 100,
    variacion_autos - 100,
    variacion_monthly_vuelos - 100,
]

etiquetas = [
    "Variación Mi Neto",
    "Inflación Acumulada",
    "Variación Alquileres",
    "Variación Big Mac",
    "Variación Autos",
    "Variación Vuelos",
]

combined = list(zip(variaciones, etiquetas))
combined.sort(key=lambda x: x[0], reverse=True)
variaciones, etiquetas = zip(*combined)

color = ["brown", "green", "orange", "red", "blue", "yellow"]

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(
    etiquetas,
    variaciones,
    color=[
        color[index] if value != "Variación Mi Neto" else "purple"
        for index, value in enumerate(etiquetas)
    ],
)

for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{height:.2f}%",
        ha="center",
        va="bottom",
    )

show_plot(
    fig,
    "Categorías",
    "Variación Porcentual (%)",
    "Variación Porcentual de Diferentes Bienes 11/2023 a Presente",
)